## BUAD 313 - Spring 2025
### Refinery Optimization

This LP is based on the refinery optimization problem described in the mini-case reading, which is available on Brightspace. Please refer to that pdf for the full details of the problem -- it is quite detailed for just two pages! 

The first cell imports the packages we always use for optimization problems.

In [2]:
import numpy as np
from gurobipy import Model, GRB, quicksum

This next cell introduces the model and defines the decision variables. Because there are many variables, it also prints the number of variables and the names of the variables so that we can check that we have all the variables we wrote down in our model. *However, it is not necessary to print the number or names of the variables in your implementation, and you can remove this code if you like.*

In [3]:
# define a gurobipy model m and name it "refinery"
m = Model("refinery")

# VARIABLES

# add linear decision variables for the input of distillation process: CrudeA and CrudeB
CrudeA = m.addVar(vtype=GRB.CONTINUOUS, name="CrudeA")
CrudeB = m.addVar(vtype=GRB.CONTINUOUS, name="CrudeB")

# add linear decision variables for the output of distillation process: LightN, MedN, HeavyN, LightO, HeavyO, Res
LightN = m.addVar(vtype=GRB.CONTINUOUS, name="LightN")
MedN = m.addVar(vtype=GRB.CONTINUOUS, name="MedN")
HeavyN = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyN")
LightO = m.addVar(vtype=GRB.CONTINUOUS, name="LightO")
HeavyO = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyO")
Res = m.addVar(vtype=GRB.CONTINUOUS, name="Res")

# add linear decision variables for the inputs of the reforming process: LightN_ReformedG, MedN_ReformedG, HeavyN_ReformedG
LightN_ReformedG = m.addVar(vtype=GRB.CONTINUOUS, name="LightN_ReformedG")
MedN_ReformedG = m.addVar(vtype=GRB.CONTINUOUS, name="MedN_ReformedG")
HeavyN_ReformedG = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyN_ReformedG")

# add a linear decision variable for the output of the reforming process: ReformedG
ReformedG = m.addVar(vtype=GRB.CONTINUOUS, name="ReformedG")

# add linear decision variables for the inputs of the cracking process: LightO_Crack, HeavyO_Crack
LightO_Crack = m.addVar(vtype=GRB.CONTINUOUS, name="LightO_Crack")
HeavyO_Crack = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyO_Crack")

# add linear decision variables for the outputs of the cracking process: CrackedG, CrackedO
CrackedG = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedG")
CrackedO = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedO")

# add linear decision variables for the inputs of the blending process for PMF: LightN_PMF, MedN_PMF, HeavyN_PMF, ReformedG_PMF, CrackedG_PMF
LightN_PMF = m.addVar(vtype=GRB.CONTINUOUS, name="LightN_PMF")
MedN_PMF = m.addVar(vtype=GRB.CONTINUOUS, name="MedN_PMF")
HeavyN_PMF = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyN_PMF")
ReformedG_PMF = m.addVar(vtype=GRB.CONTINUOUS, name="ReformedG_PMF")
CrackedG_PMF = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedG_PMF")

# add a linear decision variable for the output of the blending process for PMF: PMF
PMF = m.addVar(vtype=GRB.CONTINUOUS, name="PMF")

# add linear decision variables for the inputs of the blending process for RMF: LightN_RMF, MedN_RMF, HeavyN_RMF, ReformedG_RMF, CrackedG_RMF
LightN_RMF = m.addVar(vtype=GRB.CONTINUOUS, name="LightN_RMF")
MedN_RMF = m.addVar(vtype=GRB.CONTINUOUS, name="MedN_RMF")
HeavyN_RMF = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyN_RMF")
ReformedG_RMF = m.addVar(vtype=GRB.CONTINUOUS, name="ReformedG_RMF")
CrackedG_RMF = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedG_RMF")

# add a linear decision variable for the output of the blending process for RMF: RMF
RMF = m.addVar(vtype=GRB.CONTINUOUS, name="RMF")
# add linear decision variables for the inputs of the blending process for JF: LightO_JF, HeavyO_JF, CrackedO_JF, Res_JF
LightO_JF = m.addVar(vtype=GRB.CONTINUOUS, name="LightO_JF")
HeavyO_JF = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyO_JF")
CrackedO_JF = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedO_JF")
Res_JF = m.addVar(vtype=GRB.CONTINUOUS, name="Res_JF")

# add a linear decision variable for the output of the blending process for JF: JF
JF = m.addVar(vtype=GRB.CONTINUOUS, name="JF")

# add linear decision variables for the inputs of the blending process for FO: LightO_FO, HeavyO_FO, CrackedO_FO, Res_FO
LightO_FO = m.addVar(vtype=GRB.CONTINUOUS, name="LightO_FO")
HeavyO_FO = m.addVar(vtype=GRB.CONTINUOUS, name="HeavyO_FO")
CrackedO_FO = m.addVar(vtype=GRB.CONTINUOUS, name="CrackedO_FO")
Res_FO = m.addVar(vtype=GRB.CONTINUOUS, name="Res_FO")

# add a linear decision variable for the output of the blending process for FO: FO
FO = m.addVar(vtype=GRB.CONTINUOUS, name="FO")

# add a linear decision variable for the input for the process to create LBO: Res_LBO
Res_LBO = m.addVar(vtype=GRB.CONTINUOUS, name="Res_LBO")

# add a linear decision variable for the output of the process to create LBO: LBO
LBO = m.addVar(vtype=GRB.CONTINUOUS, name="LBO")

# update the model to include the variables
m.update()

# print the number of decision variables in the model
print("Number of decision variables in the model = ", m.NumVars)
# print the names of the variables in the model, with one variable per line and numbers in front of the variables, starting at 1
for i, v in enumerate(m.getVars()):
    print(i+1, v.varName)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-03
Number of decision variables in the model =  40
1 CrudeA
2 CrudeB
3 LightN
4 MedN
5 HeavyN
6 LightO
7 HeavyO
8 Res
9 LightN_ReformedG
10 MedN_ReformedG
11 HeavyN_ReformedG
12 ReformedG
13 LightO_Crack
14 HeavyO_Crack
15 CrackedG
16 CrackedO
17 LightN_PMF
18 MedN_PMF
19 HeavyN_PMF
20 ReformedG_PMF
21 CrackedG_PMF
22 PMF
23 LightN_RMF
24 MedN_RMF
25 HeavyN_RMF
26 ReformedG_RMF
27 CrackedG_RMF
28 RMF
29 LightO_JF
30 HeavyO_JF
31 CrackedO_JF
32 Res_JF
33 JF
34 LightO_FO
35 HeavyO_FO
36 CrackedO_FO
37 Res_FO
38 FO
39 Res_LBO
40 LBO


*From this point forward, we will complete the model together in class.*